# PyOR Quantum
## Author: Vineeth Thalakottoor
## Introduction to Quadrupole Hamiltonians (Under testing)

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/Source'

# Add source path
import sys
sys.path.append(SourcePath)

import numpy as np
# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_QuantumLibrary import QuantumLibrary
import PyOR_SphericalTensors as ST
import PyOR_Rotation as Rot
QLib = QuantumLibrary()

In [2]:
# Define the spin system
Spin_list = {"A" : "H2", "B" : "H1"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Set parameters

In [3]:
# Master Equation
QS.PropagationSpace = "Hilbert"
QS.MasterEquation = "Redfield"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = 9.4

# Offset Frequency in rotating frame (Hz)
QS.OFFSET["A"] = 10.0
QS.OFFSET["B"] = 50.0

# Define initial and final Spin Temperature
QS.I_spintemp["A"] = 300.0
QS.I_spintemp["B"] = 300.0
QS.F_spintemp["A"] = 300.0
QS.F_spintemp["B"] = 300.0

# Relaxation Process
QS.Rprocess = "Phenomenological"
QS.R1 = 1
QS.R2 = 2

QS.Update()

Rotating frame frequencies:  {'A': -386011000.0, 'B': -2514706800.0}
Offset frequencies:  {'A': 10.0, 'B': 50.0}
Initial spin temperatures:  {'A': 300.0, 'B': 300.0}
Final spin temperatures:  {'A': 300.0, 'B': 300.0}
Radiation damping gain:  {'A': 0, 'B': 0}
Radiation damping phase:  {'A': 0, 'B': 0}

Rprocess = Phenomenological

RelaxParDipole_tau = 0.0

DipolePairs = []

RelaxParDipole_bIS = []


### Zeeman Hamiltonians

In [4]:
# generate Larmor Frequencies
QS.print_Larmor = True
Ham = Hamiltonian(QS)

Larmor Frequency in MHz:  [ -61.43556874 -400.22806765]


In [5]:
# Lab Frame Hamiltonian
Hz_lab = Ham.Zeeman()
Hz_lab.Inverse2PI().matrix

Matrix([
[-261549600.665475,                0,                0,               0,                 0,                0],
[                0, 138678470.457388,                0,               0,                 0,                0],
[                0,                0, -200114040.65439,               0,                 0,                0],
[                0,                0,                0, 200114040.65439,                 0,                0],
[                0,                0,                0,               0, -138678470.457388,                0],
[                0,                0,                0,               0,                 0, 261549600.665475]])

In [6]:
# Rotating Frame Hamiltonian
Hz = Ham.Zeeman_RotFrame()
Hz.Inverse2PI().matrix

Matrix([
[-34.9999996838169,                0,                 0,                0,                 0,                0],
[                0, 15.0000012522139,                 0,                0,                 0,                0],
[                0,                0, -25.0000004680154,                0,                 0,                0],
[                0,                0,                 0, 25.0000004680154,                 0,                0],
[                0,                0,                 0,                0, -15.0000012522139,                0],
[                0,                0,                 0,                0,                 0, 34.9999996838169]])

### Quadrupole tensor PAF

In [7]:
eq = 1
etaQ = 0.5

A = "A"

IT_PAF = Ham.InteractionTensor_PAF_Quadrupole(A,eq,etaQ)
IT_PAF.Inverse2PI().matrix

Matrix([
[-16186.0646405342,                0,                0],
[                0, 21581.4195207122,                0],
[                0,                0, 21581.4195207122]])

In [8]:
PAF_Decom = Ham.InteractionTensor_PAF_Decomposition(IT_PAF)
PAF_Decom

{'Isotropic': (56500.024183590765+0j),
 'Anisotropy': (79100.03385702708+0j),
 'Asymmetry': (3+0j)}

### Quadrupole tensor LAB

In [9]:
phi,theta = 0.0,0.0

IT_LAB = Ham.InteractionTensor_LAB(IT_PAF,phi,theta,0)
IT_LAB.Round(3).matrix

Matrix([
[-101700.044,          0,          0],
[          0, 135600.058,          0],
[          0,          0, 135600.058]])

In [10]:
LAB_Decom = Ham.InteractionTensor_LAB_Decomposition(IT_LAB)
LAB_Decom['Isotropic'].matrix

Matrix([
[56500.0241835908,                0,                0],
[               0, 56500.0241835908,                0],
[               0,                0, 56500.0241835908]])

In [11]:
LAB_Decom["Symmetric"].Inverse2PI().Round(3).matrix

Matrix([
[-16186.065,        0,        0],
[         0, 21581.42,        0],
[         0,        0, 21581.42]])

In [12]:
LAB_Decom["Antisymmetric"].Inverse2PI().Round(3).matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

### Quadrupole Hamiltonian in Lab frame (Secular)

In [13]:
Hdd_LAB_Truncates = Ham.Interaction_Hamiltonian_PAF_Quadrupole_Secular(A, eq, etaQ,theta,phi)
Hdd_LAB_Truncates.Inverse2PI().Round(3).matrix

Matrix([
[10790.71,        0,          0,          0,        0,        0],
[       0, 10790.71,          0,          0,        0,        0],
[       0,        0, -21581.419,          0,        0,        0],
[       0,        0,          0, -21581.419,        0,        0],
[       0,        0,          0,          0, 10790.71,        0],
[       0,        0,          0,          0,        0, 10790.71]])

### Quadrupole Hamiltonian in Lab frame (Cartesian, Euler rotation)

In [14]:
Hdd_LAB_Euler = Ham.Interaction_Hamiltonian_Catesian_SphericalAngles(A,IT_PAF,A,theta,phi)
Hdd_LAB_Euler.Inverse2PI().Round(3).matrix

Matrix([
[ 24279.097,          0,        0,        0, -18883.741,          0],
[         0,  24279.097,        0,        0,          0, -18883.741],
[         0,          0, 5395.355,        0,          0,          0],
[         0,          0,        0, 5395.355,          0,          0],
[-18883.741,          0,        0,        0,  24279.097,          0],
[         0, -18883.741,        0,        0,          0,  24279.097]])

### Quadrupole Hamiltonian in Lab frame (Cartesian, Wigner rotation)

In [15]:
Hdd_LAB_Wig = Ham.Interaction_Hamiltonian_Catesian_Wigner(A,IT_PAF,A,phi,theta,0)
Hdd_LAB_Wig.Inverse2PI().Round(3).matrix

Matrix([
[ 24279.097,          0,        0,        0, -18883.741,          0],
[         0,  24279.097,        0,        0,          0, -18883.741],
[         0,          0, 5395.355,        0,          0,          0],
[         0,          0,        0, 5395.355,          0,          0],
[-18883.741,          0,        0,        0,  24279.097,          0],
[         0, -18883.741,        0,        0,          0,  24279.097]])

### Quadrupole Hamiltonian in Lab frame (Spherical Tensors, Wigner Rotation)

In [16]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","all", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[ 24279.097,          0,        0,        0, -18883.741,          0],
[         0,  24279.097,        0,        0,          0, -18883.741],
[         0,          0, 5395.355,        0,          0,          0],
[         0,          0,        0, 5395.355,          0,          0],
[-18883.741,          0,        0,        0,  24279.097,          0],
[         0, -18883.741,        0,        0,          0,  24279.097]])

In [17]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","secular", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[24279.097,         0,        0,        0,         0,         0],
[        0, 24279.097,        0,        0,         0,         0],
[        0,         0, 5395.355,        0,         0,         0],
[        0,         0,        0, 5395.355,         0,         0],
[        0,         0,        0,        0, 24279.097,         0],
[        0,         0,        0,        0,         0, 24279.097]])

In [18]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","secular + pseudosecular", phi,theta,0)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[24279.097,         0,        0,        0,         0,         0],
[        0, 24279.097,        0,        0,         0,         0],
[        0,         0, 5395.355,        0,         0,         0],
[        0,         0,        0, 5395.355,         0,         0],
[        0,         0,        0,        0, 24279.097,         0],
[        0,         0,        0,        0,         0, 24279.097]])